In [1]:
import pandas as pd
import json

from utils.groq_chat import GroqChat

In [2]:
df=pd.read_csv("../sample_data/data1.csv")

In [3]:
df

,id,first_name,last_name,email,gender,phone,Channel,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Region
0,1,Chas,Rollett,crollett0@icq.com,Male,625-741-0213,2,12669,9656,7561,214,2674,1338,3
1,2,Rockie,Cheney,rcheney1@delicious.com,Male,233-712-5425,2,7057,9810,9568,1762,3293,1776,3
2,3,Gretel,Pole,gpole2@elpais.com,Female,514-912-5355,2,6353,8808,7684,2405,3516,7844,3
3,4,Bertine,Scogings,bscogings3@bizjournals.com,Female,622-446-1554,1,13265,1196,4221,6404,507,1788,3
4,5,Leonid,Shortcliffe,lshortcliffe4@ted.com,Male,384-201-2900,2,22615,5410,7198,3915,1777,5185,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,436,Valery,Ternott,vternottc3@bandcamp.com,Female,238-620-6990,1,29703,12051,16027,13135,182,2204,3
436,437,Roxy,Forsyde,rforsydec4@nymag.com,Female,945-246-6828,1,39228,1431,764,4510,93,2346,3
437,438,Jess,Skrines,jskrinesc5@wikia.com,Genderfluid,936-756-5245,2,14531,15488,30243,437,14841,1867,3
438,439,Burke,Sallan,bsallanc6@wp.com,Male,680-783-4295,1,10290,1981,2232,1038,168,2125,3


In [4]:
local_dataset=pd.DataFrame(columns=["Name","Contact","Email","Info","location","Product_to_pitch"])

In [5]:

# Load JSON from a file
with open("./utils/mapping_rules.json", "r") as file:
    rules = json.load(file)

# Print the loaded JSON
print(rules)


{'mapping_rules': {'Name': {'description': 'Represents the full name of a person, which can include first and last name. If only one part of the name is available, it should be used as is.'}, 'Contact': {'description': 'Represents a phone number or any contact number of the person. The number should be in a standardized format.'}, 'Email': {'description': 'Represents the email address of the person. It should be in a valid email format.'}, 'Info': {'description': 'Contains any additional relevant information, remarks, or general details about the person that do not fit into other fields.'}, 'Location': {'description': 'Represents the geographical location of the person, which may include city, state, country, or a full address.'}, 'Product_to_pitch': {'description': 'Represents the product or service that should be pitched to the person based on their interests or preferences.'}}}


In [6]:
from rich import print as p

p(rules)

{
    'mapping_rules': {
        'Name': {
            'description': 'Represents the full name of a person, which can include first and last name. If only 
one part of the name is available, it should be used as is.'
        },
        'Contact': {
            'description': 'Represents a phone number or any contact number of the person. The number should be in 
a standardized format.'
        },
        'Email': {
            'description': 'Represents the email address of the person. It should be in a valid email format.'
        },
        'Info': {
            'description': 'Contains any additional relevant information, remarks, or general details about the 
person that do not fit into other fields.'
        },
        'Location': {
            'description': 'Represents the geographical location of the person, which may include city, state, 
country, or a full address.'
        },
        'Product_to_pitch': {
            'description': 'Represents the product or service that should be pitched to the person based on their 
interests or preferences.'
        }
    }
}

In [7]:
llm=GroqChat()


In [12]:
def get_dataset_described(df_head:pd.DataFrame):
    system_prompt=f"""You are a data analysis expert. When given a pandas DataFrame's head output, carefully analyze each column and provide a comprehensive yet concise description. Your response must be a valid JSON object where:
        - Keys are column names
        - Values are detailed descriptions capturing the column's nature, potential meaning, and data characteristics
        - Descriptions should be precise, informative, and max 100 characters long"""

    user_prompt=f"""Analyze the following DataFrame head and provide a JSON description of each column's characteristics and potential meaning:
        {df_head.to_string}
        Format your response as a valid JSON object with descriptive insights for each column."""
    response=llm.simple_chat(system_prompt=system_prompt,user_prompt=user_prompt)
    return response.split("```json")[-1]


In [13]:
report= get_dataset_described(df.head())
print(report)


{
    "id": {
        "nature": "Integer",
        "potential_meaning": "Unique identifier for each customer or record",
        "data_characteristics": "Sequential, non-negative integers"
    },
    "first_name": {
        "nature": "String",
        "potential_meaning": "Customer's given name",
        "data_characteristics": "Human-readable, case-sensitive, varying lengths"
    },
    "last_name": {
        "nature": "String",
        "potential_meaning": "Customer's surname",
        "data_characteristics": "Human-readable, case-sensitive, varying lengths"
    },
    "email": {
        "nature": "String",
        "potential_meaning": "Customer's email address for communication",
        "data_characteristics": "Human-readable, case-sensitive, varying lengths"
    },
    "gender": {
        "nature": "Categorical",
        "potential_meaning": "Customer's gender (Male/Female)",
        "data_characteristics": "Discrete, binary values"
    },
    "phone": {
        "nature": "String

In [14]:
def check_column_compatibility_using_rules(column_name:str,column_desc:str,dataset:json):
    system_prompt="""You are an expert data analyst who maps data across different datasets and your task is to determine the best match for a target column from the source dataset based on their descriptions that can it be mapped or not. 
    You will be given the target column and its description along with multiple source columns and their descriptions.

Your response should follow these rules:
- Compare the **target column's description** with the **descriptions of all source columns**.
- If the descriptions of the source columns align with the target column, return the **name of the source column** that best matches the target column.
- If no source column is a good match for the target column, return "Nothing Compatible".
- The descriptions and columns should be analyzed for **compatibility** based on their relevance and meaning.

Return **only** the name of the matching column or "Nothing Compatible" if no match is found.

The format of the input will be:
- One **target column**
- Multiple **source columns** with their respective descriptions.

Do not include any other text besides the matching column name or "Nothing Compatible".
"""

    user_prompt=F"""The target column and its description are as follows:
    Target Column: {column_name}
    Description: {column_desc}

    Here are the source columns with their descriptions:
    
    {json.dumps(dataset)}
    
    """
    response=llm.simple_chat(user_prompt=user_prompt,system_prompt=system_prompt)
    return response



In [35]:
data = rules["mapping_rules"]
keys, values = zip(*data.items())  # This unpacks the items into two separate tuples
values[0]


{'description': 'Represents the full name of a person, which can include first and last name. If only one part of the name is available, it should be used as is.'}

In [37]:
response=check_column_compatibility_using_rules(column_name=keys[0],column_desc=values[0]["description"],dataset=report)

print(response)

first_name
last_name


In [40]:
print(df.head())

   id first_name    last_name                       email  gender  \
0   1       Chas      Rollett           crollett0@icq.com    Male   
1   2     Rockie       Cheney      rcheney1@delicious.com    Male   
2   3     Gretel         Pole           gpole2@elpais.com  Female   
3   4    Bertine     Scogings  bscogings3@bizjournals.com  Female   
4   5     Leonid  Shortcliffe       lshortcliffe4@ted.com    Male   

          phone  Channel  Fresh  Milk  Grocery  Frozen  Detergents_Paper  \
0  625-741-0213        2  12669  9656     7561     214              2674   
1  233-712-5425        2   7057  9810     9568    1762              3293   
2  514-912-5355        2   6353  8808     7684    2405              3516   
3  622-446-1554        1  13265  1196     4221    6404               507   
4  384-201-2900        2  22615  5410     7198    3915              1777   

   Delicassen  Region  
0        1338       3  
1        1776       3  
2        7844       3  
3        1788       3  
4       